In [1]:
from glob import glob
import pandas as pd
import re
from pprint import pprint
import pkg_resources

from pymedextcore.document import Document
from pymedext_eds.annotators import Endlines, SentenceTokenizer, SectionSplitter
from pymedext_eds.med import MedicationAnnotator, MedicationNormalizer

In [2]:
models_param = [{'tagger_path':'/export/home/cse180025/prod_information_extraction/data/models/apmed4/entities/final-model.pt' ,
                'tag_name': 'entity_pred' },
                {'tagger_path':'/export/home/cse180025/prod_information_extraction/data/models/apmed4/events/final-model.pt' ,
                'tag_name': 'event_pred' },
               {'tagger_path': "/export/home/cse180025/prod_information_extraction/data/models/apmed4/drugblob/final-model.pt",
                'tag_name': 'drugblob_pred'}]

data_path = pkg_resources.resource_filename('pymedext_eds', 'data/romedi')
romedi_path = glob(data_path + '/*.p')[0]

In [3]:
endlines = Endlines(["raw_text"], "clean_text", ID="endlines")
sections = SectionSplitter(['clean_text'], "section", ID= 'sections')
sentenceSplitter = SentenceTokenizer(["section"],"sentence", ID="sentences")
med = MedicationAnnotator(['sentence'], 'med', ID='med:v2', models_param=models_param,  device='cuda:1')
norm = MedicationNormalizer(['ENT/DRUG','ENT/CLASS'], 'normalized_mention', ID='norm',romedi_path= romedi_path)

2020-11-05 14:43:38,207 loading file /export/home/cse180025/prod_information_extraction/data/models/apmed4/entities/final-model.pt
2020-11-05 14:43:42,301 loading file /export/home/cse180025/prod_information_extraction/data/models/apmed4/events/final-model.pt
2020-11-05 14:43:43,925 loading file /export/home/cse180025/prod_information_extraction/data/models/apmed4/drugblob/final-model.pt


In [4]:
data_path = pkg_resources.resource_filename('pymedext_eds', 'data/demo')
file_list = glob(data_path + '/*.txt')

docs = []
for docid in file_list:

    with open(docid) as f: 
        txt = f.read()
    
    docs.append(Document(txt,ID = docid, attributes= {'person_id' : '1234'}))

[x.annotate([endlines,sections, sentenceSplitter, med, norm]) for x in docs]

Ignore 2 sentence(s) with no tokens.


[None, None]

In [5]:
pprint(MedicationAnnotator.doc_to_omop(docs[1]))

[{'lexical_variant': 'Pyostacine',
  'nlp_date': '2020-11-05',
  'nlp_datetime': '2020-11-05 14:43:48',
  'nlp_system': 'medext_v3',
  'note_id': '/export/home/cse180025/prod_information_extraction/pymedext_eds/pymedext_eds/data/demo/cas2.txt',
  'note_nlp_concept_id': 'J01FG01',
  'note_nlp_id': None,
  'note_nlp_source_concept_id': 'ATC',
  'offset_begin': 65,
  'offset_end': 75,
  'person_id': '1234',
  'section_concept_id': 'head',
  'snippet': 'Le 9 décembre 2010, une femme de 79 ans reçoit un traitement par '
             'Pyostacine® pour, semble-t-il, un ulcère variqueux infecté',
  'term_exists': None,
  'term_modifiers': "section='head',score='0.9963575005531311',snippet='reçoit "
                    "un traitement par Pyostacine',ATC_LABEL='PRISTINAMYCINE'",
  'term_temporal': None,
  'validation_level_id': 'automated'},
 {'lexical_variant': 'tulle gras',
  'nlp_date': '2020-11-05',
  'nlp_datetime': '2020-11-05 14:43:48',
  'nlp_system': 'medext_v3',
  'note_id': '/export/h